###Build Data Pipeline using Open weather data

In [1]:
#import libraries
import json
import requests
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
api.key= 'api_kk'
city= "Tampere,FI"

url= f"http://api.openweathermap.org/data/2.5/forecast?q={city}&cnt=240&appid={api.key}"
response=requests.get(url)

In [5]:
#Extract the data
data=response.json()
json_str= json.dumps(data,indent=4)
print(json_str)

{
    "cod": "200",
    "message": 0,
    "cnt": 40,
    "list": [
        {
            "dt": 1742007600,
            "main": {
                "temp": 268.17,
                "feels_like": 262.37,
                "temp_min": 268.17,
                "temp_max": 269.72,
                "pressure": 1009,
                "sea_level": 1009,
                "grnd_level": 994,
                "humidity": 86,
                "temp_kf": -1.55
            },
            "weather": [
                {
                    "id": 801,
                    "main": "Clouds",
                    "description": "few clouds",
                    "icon": "02n"
                }
            ],
            "clouds": {
                "all": 14
            },
            "wind": {
                "speed": 4.48,
                "deg": 242,
                "gust": 13.92
            },
            "visibility": 10000,
            "pop": 0,
            "sys": {
                "pod": "n"
            },
        

##TRANSFORMATION FROM JSON TO DATAFRAME

In [8]:
#convert temperature from kelvin to celsuis

def kelvin_to_celsius(temp_k):
    return temp_k - 273.15

In [10]:
if response.status_code == 200:
    data = response.json()
     # Check if 'list' key is present
    if 'list' not in data:
        print("Error: 'list' key missing in API response.")
        print(data)  # Print full response for debugging
    else:
        weather_data = []

    for hour in data['list']:
        date_time = datetime.utcfromtimestamp(hour['dt']).strftime('%Y-%m-%d %H:%M:%S')
        temp = kelvin_to_celsius(hour['main']['temp'])
        feels_like = kelvin_to_celsius(hour['main']['feels_like'])
        pressure = hour['main']['pressure']
        humidity = hour['main']['humidity']
        wind_speed = hour['wind']['speed']
        wind_direction = hour['wind']['deg']
        cloudiness = hour['clouds']['all']
        weather_main = hour['weather'][0]['main']
        weather_description = hour['weather'][0]['description']
        rain_volume = hour.get('rain', {}).get('3h', 0)
        snow_volume = hour.get('snow', {}).get('3h', 0)

   
        weather_data.append({
            "DateTime": date_time,
            "Temperature": temp,
            "Feels Like_temp" : feels_like,
            "Pressure(hPa)" : pressure,
            "Humidity_percent" : humidity,
            "Weather" : weather_main,
            "Weather Description" : weather_description,
            "Wind Speed": wind_speed,
            "Wind Direction": wind_direction,
            "Cloudiness": cloudiness,
            "Rain Volume(mm)": rain_volume,
            "Snow Volume(mm)": snow_volume
        })
        df = pd.DataFrame(weather_data)
else:
    prrint(f"Failed to get data : {response.status_code}")

df.head()

,DateTime,Temperature,Feels Like_temp,Pressure(hPa),Humidity_percent,Weather,Weather Description,Wind Speed,Wind Direction,Cloudiness,Rain Volume(mm),Snow Volume(mm)
0,2025-03-15 03:00:00,-4.98,-10.78,1009,86,Clouds,few clouds,4.48,242,14,0.0,0.0
1,2025-03-15 06:00:00,-2.79,-8.99,1009,77,Clouds,scattered clouds,6.00,243,34,0.0,0.0
2,2025-03-15 09:00:00,2.56,-2.70,1008,60,Clouds,broken clouds,7.20,247,67,0.0,0.0
3,2025-03-15 12:00:00,2.97,-2.25,1007,53,Clouds,broken clouds,7.41,248,83,0.0,0.0
4,2025-03-15 15:00:00,1.31,-4.39,1005,60,Clouds,overcast clouds,7.39,245,100,0.0,0.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DateTime             40 non-null     object 
 1   Temperature          40 non-null     float64
 2   Feels Like_temp      40 non-null     float64
 3   Pressure(hPa)        40 non-null     int64  
 4   Humidity_percent     40 non-null     int64  
 5   Weather              40 non-null     object 
 6   Weather Description  40 non-null     object 
 7   Wind Speed           40 non-null     float64
 8   Wind Direction       40 non-null     int64  
 9   Cloudiness           40 non-null     int64  
 10  Rain Volume(mm)      40 non-null     float64
 11  Snow Volume(mm)      40 non-null     float64
dtypes: float64(5), int64(4), object(3)
memory usage: 3.9+ KB


LOADING OF THE DATA INTO POSTGRESQL DATABASE

In [15]:
import psycopg2 # for connecting to postgresql
from sqlalchemy import create_engine #tof efficiently reuse the connection

In [17]:
db_username = 'postgres'
db_password = 'PASSWORD'
db_host = 'localhost'
db_port = 5432
db_name = 'postgres'

In [19]:
import urllib.parse
encoded_password = urllib.parse.quote(db_password)
engine = create_engine(f'postgresql://{db_username}:{encoded_password}@{db_host}:{db_port}/{db_name}')


In [21]:
table_name = 'weather_data'

df.to_sql(table_name, engine, if_exists='replace', index=False)

engine.dispose()